In [1]:
import os
import numpy as np
import codecs
path_baptiste = "/home/baptiste/Documents/data/train"
path_igor = "C:/Users/Igor/Documents/Master Data Science/Big Data Analytics/Projet/Data/train"
path_igor_test = "C:/Users/Igor/Documents/Master Data Science/Big Data Analytics/Projet/Data/test"
path_sofia = "/Users/Flukmacdesof/data 2/train"



#assumes labelled data ra stored into a positive and negative folder
#returns two lists one with the text per file and another with the corresponding class 
def loadLabeled(path):

	rootdirPOS =path+'/pos'
	rootdirNEG =path+'/neg'
	data=[]
	Class=[]
	count=0
	for subdir, dirs, files in os.walk(rootdirPOS):
		
		for file in files:
			with codecs.open(rootdirPOS+"/"+file, 'r',encoding="utf-8") as content_file:
				content = content_file.read() #assume that there are NO "new line characters"
				data.append(content)
	tmpc1=np.ones(len(data))
	for subdir, dirs, files in os.walk(rootdirNEG):
		
		for file in files:
			with codecs.open(rootdirNEG+"/"+file, 'r',encoding="utf-8") as content_file:
				content = content_file.read() #assume that there are NO "new line characters"
				data.append(content)
	tmpc0=np.zeros(len(data)-len(tmpc1))
	Class=np.concatenate((tmpc1,tmpc0),axis=0)
	return data,Class
#loads unlabelled data	
#returns two lists
#one with the data per file and another with the respective filenames (without the file extension)
def loadUnknown(path):
	rootdir=path
	data=[]
	names=[]
	for subdir, dirs, files in os.walk(rootdir):
		for file in files:
			with codecs.open(rootdir+"/"+file, 'r', encoding= "utf-8") as content_file:
				content = content_file.read() #assume that there are NO "new line characters"
				data.append(content)
				names.append(file.split(".")[0])
	return data,names

In [2]:
reviews, Class = loadLabeled(path_igor)
test_test_reviews, names = loadUnknown(path_igor_test)

In [3]:
print(1)

1


In [10]:
import re
import nltk
from bs4 import BeautifulSoup
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]

        b=[]
        stemmer = english_stemmer #PorterStemmer()
        for word in words:
            b.append(stemmer.stem(word))

        # 5. Return a list of words
        return(b)

In [8]:
from sklearn.cross_validation import train_test_split
train_reviews, test_reviews, train_label, test_label = train_test_split(reviews, Class, test_size = 0.25, random_state = 42)

In [11]:
clean_train_reviews = []
for review in train_reviews:
    clean_train_reviews.append( " ".join( review_to_wordlist( review, remove_stopwords = False )))
    
clean_test_reviews = []
for review in test_reviews:
    clean_test_reviews.append( " ".join( review_to_wordlist( review, remove_stopwords = False )))

C:\Users\Igor\Anaconda\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer( min_df=2, max_df=0.95, max_features = 200000, ngram_range = ( 1, 4 ), sublinear_tf = True )

train_data_features = vectorizer.fit_transform(clean_train_reviews)
test_data_features = vectorizer.transform( clean_test_reviews )

In [13]:
print("Reducing dimension...")

from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
fselect = SelectKBest(chi2 , k=70000)
train_data_features = fselect.fit_transform(train_data_features, train_label)
test_data_features = fselect.transform(test_data_features)

Reducing dimension...


In [14]:
1

1

In [15]:
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

model1 = LogisticRegression(penalty = 'l2')
model1.fit(train_data_features, train_label)

model2 = SGDClassifier(loss='modified_huber', n_iter=5, random_state=0, shuffle=True)
model2.fit( train_data_features, train_label )

p1 = model1.predict_proba( test_data_features )[:,1]
p2 = model2.predict_proba( test_data_features )[:,1]

In [16]:
def score(true_label, predicted_label):
    length = len(true_label)
    total = 0
    for i, label in enumerate(true_label):
        if label == predicted_label[i]:
            total += 1
    return float(total)/float(length)

In [17]:
Cs = np.linspace(0,10, 101)
bestC = -1
bestScore = 0
for C in Cs:
    proba = C*p1 + 1.* p2
    predicted_label = [1 if p/(1.+C)>0.5 else 0 for p in proba]
    if score(test_label, predicted_label) > bestScore:
        bestScore = score(test_label, predicted_label)
        bestC = C

proba = bestC*p1 + 1.* p2
predicted_label = [1 if p >0.5 else 0 for p in proba]
print(bestC, score(test_label, predicted_label))

0.0 0.91008


In [18]:
predicted_label = model2.predict(test_data_features)
print(score(test_label, predicted_label))

0.91008


In [22]:
train_reviews[3]

'This review is for the extended cut of this movie.<br /><br />I first watched Dragon Lord when I bought it on DVD many years ago. I always liked this movie and you can read some of the more positive reviews of it to get the general idea.<br /><br />That being said. I\'ve always found the storyline a bit confusing. The movie is, after all, a love story. And it always seemed strange to me that a love story should end with a 20 minute fight scene.<br /><br />Well, in the extended version this is no longer so. The old "original" version begins off with a huge barrel-climb/rugby-like sequence which is the new ending sequence in the extended version. The opening sequence is Dragon(Jackie Chan) hanging around his house and pretending to be training and reciting whenever his father is around.<br /><br />Other sequenced have also been shift or prolonged in the extended cut and the story makes a lot more sense when you watch it. The pacing is also better and overall it just works better. It fee

In [23]:
clean_train_reviews[3]

'this review is for the extend cut of this movi i first watch dragon lord when i bought it on dvd mani year ago i alway like this movi and you can read some of the more posit review of it to get the general idea that be said i ve alway found the storylin a bit confus the movi is after all a love stori and it alway seem strang to me that a love stori should end with a minut fight scene well in the extend version this is no longer so the old origin version begin off with a huge barrel climb rugbi like sequenc which is the new end sequenc in the extend version the open sequenc is dragon jacki chan hang around his hous and pretend to be train and recit whenev his father is around other sequenc have also been shift or prolong in the extend cut and the stori make a lot more sens when you watch it the pace is also better and overal it just work better it feel more like a love stori and doesn t leav you ask question about whi it end so drastic and dramat as the regular version doe i suggest ev

In [24]:
review_to_wordlist(train_reviews[3])

C:\Users\Igor\Anaconda\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


['this',
 'review',
 'is',
 'for',
 'the',
 'extend',
 'cut',
 'of',
 'this',
 'movi',
 'i',
 'first',
 'watch',
 'dragon',
 'lord',
 'when',
 'i',
 'bought',
 'it',
 'on',
 'dvd',
 'mani',
 'year',
 'ago',
 'i',
 'alway',
 'like',
 'this',
 'movi',
 'and',
 'you',
 'can',
 'read',
 'some',
 'of',
 'the',
 'more',
 'posit',
 'review',
 'of',
 'it',
 'to',
 'get',
 'the',
 'general',
 'idea',
 'that',
 'be',
 'said',
 'i',
 've',
 'alway',
 'found',
 'the',
 'storylin',
 'a',
 'bit',
 'confus',
 'the',
 'movi',
 'is',
 'after',
 'all',
 'a',
 'love',
 'stori',
 'and',
 'it',
 'alway',
 'seem',
 'strang',
 'to',
 'me',
 'that',
 'a',
 'love',
 'stori',
 'should',
 'end',
 'with',
 'a',
 'minut',
 'fight',
 'scene',
 'well',
 'in',
 'the',
 'extend',
 'version',
 'this',
 'is',
 'no',
 'longer',
 'so',
 'the',
 'old',
 'origin',
 'version',
 'begin',
 'off',
 'with',
 'a',
 'huge',
 'barrel',
 'climb',
 'rugbi',
 'like',
 'sequenc',
 'which',
 'is',
 'the',
 'new',
 'end',
 'sequenc',
 'i